In [2]:
import mysql.connector
import datetime
from datetime import datetime
import pandas as pd

In [1]:
config = {
    "user": "root",
    "password": "sua_nova_senha",
    "host": "localhost",
    "port": "3306",
    "database": "cursoCD"
}

class ConexaoBD:
    def __init__(self, config):
        self.connection = mysql.connector.connect(**config)
        self.cursor = self.connection.cursor()

    def commit_and_close(self):
        self.connection.commit()
        self.connection.close()



class Filme:
    def __init__(self, id, titulo, media_votos, contagem_votos, status_filme, data_lancamento, faturamento,
                  adulto, orcamento, id_imdb, titulo_original, resumo, popularidade):
        self.id = id
        self.titulo = titulo
        self.media_votos = media_votos
        self.contagem_votos = contagem_votos
        self.status_filme = status_filme
        self.data_lancamento = data_lancamento
        self.faturamento = faturamento
        self.adulto = adulto
        self.orcamento = orcamento
        self.id_imdb = id_imdb
        self.titulo_original = titulo_original
        self.resumo = resumo
        self.popularidade = popularidade

In [13]:
def criar_banco():
    conexao = ConexaoBD(config)
    cursor = conexao.cursor
    queries = '''


CREATE TABLE produtora (
    id INT PRIMARY KEY AUTO_INCREMENT,
    nome VARCHAR(1000) NOT NULL
);

CREATE TABLE filme (
    id INT PRIMARY KEY,
    titulo VARCHAR(1000) NOT NULL,
    media_votos FLOAT CHECK (media_votos >= 0 AND media_votos <= 10),
    contagem_votos INT DEFAULT 0,
    status_filme VARCHAR(50),
    data_lancamento DATE,
    faturamento FLOAT DEFAULT 0.00,
    adulto BOOLEAN DEFAULT FALSE,
    orcamento FLOAT DEFAULT 0.00,
    id_imdb VARCHAR(50)como,
    titulo_original VARCHAR(255),
    resumo TEXT,
    popularidade FLOAT DEFAULT 0.00
);

CREATE TABLE filme_produtora (
    id_filme INT NOT NULL,
    id_produtora INT NOT NULL,  
	foreign key (id_filme) REFERENCES filme(id) on delete cascade,
    foreign key (id_produtora) references produtora (id) on delete cascade,
    PRIMARY KEY (id_filme ,id_produtora) 
);


CREATE TABLE idioma
(
id int PRIMARY KEY AUTO_INCREMENT, 
nome varchar(100) NOT NULL
);

CREATE TABLE filme_idioma (
id_filme INTEGER NOT NULL,
id_idioma INTEGER NOT NULL,
foreign key (id_filme) references filme (id),
foreign key (id_idioma) references idioma(id),
primary key (id_filme, id_idioma)
);


CREATE TABLE genero
(
id int PRIMARY KEY AUTO_INCREMENT, 
nome varchar(100) NOT NULL
);


CREATE TABLE filme_genero (
id_filme INTEGER NOT NULL,
id_genero INTEGER NOT NULL,
foreign key (id_filme) references filme (id),
foreign key (id_genero) references genero(id),
primary key (id_filme, id_genero)
);

            '''
    for query in queries.strip().split(';'):
        if query.strip():  
            cursor.execute(query)
    conexao.commit_and_close()

In [14]:
def inserir_filme(id, titulo, media_votos, contagem_votos, status_filme, data_lancamento, faturamento, adulto, orcamento, id_imdb, titulo_original, resumo, popularidade):
    conexao = ConexaoBD(config=config)
    cursor = conexao.cursor
    query = '''
        INSERT INTO filme (
            id, titulo, media_votos, contagem_votos, status_filme, data_lancamento, 
            faturamento, adulto, orcamento, id_imdb, titulo_original, resumo, popularidade
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
    '''
    valores = (
        id, titulo, media_votos, contagem_votos, status_filme, data_lancamento, 
        faturamento, adulto, orcamento, id_imdb, titulo_original, resumo, popularidade
    )
    cursor.execute(query, valores)
    conexao.commit_and_close()


In [ ]:
def inserir_filmes(lista_filmes):
    conexao = ConexaoBD(config=config)
    cursor = conexao.cursor

    try:
        # Itera sobre a lista de filmes e insere cada um no banco de dados
        for filme in lista_filmes:
            
            query = '''
                INSERT INTO filme (
                    id, titulo, media_votos, contagem_votos, status_filme, data_lancamento, 
                    faturamento, adulto, orcamento, id_imdb, titulo_original, resumo, popularidade
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
            '''
            valores = (
                filme.id, filme.titulo, filme.media_votos, filme.contagem_votos, filme.status_filme, filme.data_lancamento,
                filme.faturamento, filme.adulto, filme.orcamento, filme.id_imdb, filme.titulo_original, filme.resumo, filme.popularidade
            )
            cursor.execute(query, valores)
            print(f'filme: {filme.titulo} id: {filme.id}')
        # Commit após todos os filmes serem inseridos
        conexao.commit_and_close()
        print("Todos os filmes foram inseridos com sucesso.")

    except mysql.connector.Error as err:
        print(f"Erro: {err}")
        conexao.connection.rollback()
    finally:
        conexao.connection.close()



In [ ]:
def inserir_produtora(nome):
    conexao = ConexaoBD(config=config)
    cursor = conexao.cursor
    query = '''
        INSERT INTO produtora (nome) VALUES (%s)
    '''
    valores = (nome.lower(),)

    try:
        cursor.execute(query, valores)
        conexao.commit_and_close()
        print("produtora inserida com sucesso.")
    except mysql.connector.Error as err:
        print(f"Erro: {err}")
        conexao.connection.rollback()
        conexao.connection.close()



In [ ]:
def inserir_produtoras(lista_produtoras):
    conexao = ConexaoBD(config=config)

    cursor = conexao.cursor

    query = '''
        INSERT INTO produtora (nome) VALUES (%s)
    '''

    try:
        # Iterar sobre a lista de nomes e inserir cada um na tabela
        for nome in lista_produtoras:
            valores = (nome.lower(),)
            cursor.execute(query, valores)
            print(f'produtora: {nome}')
        # Commit após todos os registros serem inseridos
    
        conexao.commit_and_close()
        print("Todas as produtoras foram inseridas com sucesso.")

    except mysql.connector.Error as err:
        print(f"Erro: {err}")
        conexao.connection.rollback()
    finally:
        conexao.connection.close()

In [ ]:
def inserir_filme_produtora(id_filme, id_produtora):
    conexao = ConexaoBD(config=config)
    cursor = conexao.cursor
    query = '''
        INSERT INTO filme_produtora (
            id_filme, id_produtora
        ) VALUES (%s, %s)
    '''
    valores = (id_filme, id_produtora)

    try:
        cursor.execute(query, valores)
        conexao.commit_and_close()
        print("Relação filme-produtora inserida com sucesso.")
    except mysql.connector.Error as err:
        print(f"Erro: {err}")
        conexao.connection.rollback()
        conexao.connection.close()

def inserir_filmes_produtoras(lista_relacoes):
    conexao = ConexaoBD(config=config)
    cursor = conexao.cursor
    query = '''
        INSERT INTO filme_produtora (
            id_filme, id_produtora
        ) VALUES (%s, %s)
    '''

    try:
        # Iterar sobre a lista de relações e inserir cada uma
        for relacao in lista_relacoes:
            id_filme = int(relacao[0])  # Converter para int
            id_produtora = int(relacao[1])  # Converter para int
            valores = (id_filme, id_produtora)
            
            try:
                cursor.execute(query, valores)
            except mysql.connector.Error as err:
                # Se for erro de duplicação (1062), apenas ignore e continue
                if err.errno == 1062:
                    print(f"Entrada duplicada ignorada: Filme {id_filme} - Produtora {id_produtora}")
                    continue
                else:
                    raise  # Se não for duplicado, levante o erro novamente

        # Commit após todas as inserções
        conexao.commit_and_close()
        print("Todas as relações filme-produtora foram inseridas com sucesso.")

    except mysql.connector.Error as err:
        print(f"Erro: {err}")
        conexao.connection.rollback()
    finally:
        conexao.connection.close()




In [ ]:
def inserir_idioma(nome):
    conexao = ConexaoBD(config=config)
    cursor = conexao.cursor
    query = '''
        INSERT INTO idioma (nome) VALUES (%s)
    '''
    valores = (nome,)

    try:
        cursor.execute(query, valores)
        conexao.commit_and_close()
        print("Idioma inserido com sucesso.")
    except mysql.connector.Error as err:
        print(f"Erro: {err}")
        conexao.connection.rollback()
        conexao.connection.close()




In [ ]:
def inserir_filme_idioma(id_filme, id_idioma):
    conexao = ConexaoBD(config=config)
    cursor = conexao.cursor
    query = '''
        INSERT INTO filme_idioma (
            id_filme, id_idioma
        ) VALUES (%s, %s)
    '''
    valores = (id_filme, id_idioma)

    try:
        cursor.execute(query, valores)
        conexao.commit_and_close()
        print("Relação filme-idioma inserida com sucesso.")
    except mysql.connector.Error as err:
        print(f"Erro: {err}")
        conexao.connection.rollback()
        conexao.connection.close()

def inserir_filmes_idiomas(lista_relacoes):
    conexao = ConexaoBD(config=config)
    cursor = conexao.cursor
    query = '''
        INSERT INTO filme_idioma (
            id_filme, id_idioma
        ) VALUES (%s, %s)
    '''

    try:
        # Iterar sobre a lista de relações e inserir cada uma
        for relacao in lista_relacoes:
            id_filme = int(relacao[0])  # Converter para int
            id_idioma = int(relacao[1])  # Converter para int
            valores = (id_filme, id_idioma)
            
            try:
                cursor.execute(query, valores)
            except mysql.connector.Error as err:
                # Se for erro de duplicação (1062), apenas ignore e continue
                if err.errno == 1062:
                    print(f"Entrada duplicada ignorada: Filme {id_filme} - Idioma {id_idioma}")
                    continue
                else:
                    raise  # Se não for duplicado, levante o erro novamente

        # Commit após todas as inserções
        conexao.commit_and_close()
        print("Todas as relações filme-idioma foram inseridas com sucesso.")

    except mysql.connector.Error as err:
        print(f"Erro: {err}")
        conexao.connection.rollback()
    finally:
        conexao.connection.close()


In [7]:
def inserir_genero(nome):
    conexao = ConexaoBD(config=config)
    cursor = conexao.cursor
    
    try:
        # Verificar se o gênero já existe
        cursor.execute('SELECT id FROM genero WHERE nome = %s', (nome,))
        resultado = cursor.fetchone()

        if resultado:
            print(f"O gênero '{nome}' já existe no banco de dados.")
        else:
            # Se não existir, inserir o gênero
            query = 'INSERT INTO genero (nome) VALUES (%s)'
            valores = (nome,)
            cursor.execute(query, valores)
            conexao.commit_and_close()
            print(f"Gênero '{nome}' inserido com sucesso.")
    except mysql.connector.Error as err:
        print(f"Erro: {err}")
        conexao.connection.rollback()
    finally:
        conexao.connection.close()

In [ ]:
def inserir_filme_genero(id_filme, id_genero):
    conexao = ConexaoBD(config=config)
    cursor = conexao.cursor
    query = '''
        INSERT INTO filme_genero (
            id_filme, id_genero
        ) VALUES (%s, %s)
    '''
    valores = (id_filme, id_genero)

    try:
        cursor.execute(query, valores)
        conexao.commit_and_close()
        print("Relação filme-gênero inserida com sucesso.")
    except mysql.connector.Error as err:
        print(f"Erro: {err}")
        conexao.connection.rollback()
        conexao.connection.close()



def inserir_filmes_generos(lista_relacoes):
    conexao = ConexaoBD(config=config)
    cursor = conexao.cursor
    query = '''
        INSERT INTO filme_genero (
            id_filme, id_genero
        ) VALUES (%s, %s)
    '''

    try:
        # Iterar sobre a lista de relações e inserir cada uma
        for relacao in lista_relacoes:
            id_filme = int(relacao[0])  # Converter para int
            id_genero = int(relacao[1])  # Converter para int
            valores = (id_filme, id_genero)
            
            try:
                cursor.execute(query, valores)
            except mysql.connector.Error as err:
                # Se for erro de duplicação (1062), apenas ignore e continue
                if err.errno == 1062:
                    print(f"Entrada duplicada ignorada: Filme {id_filme} - Gênero {id_genero}")
                    continue
                else:
                    raise  # Se não for duplicado, levante o erro novamente

        # Commit após todas as inserções
        conexao.commit_and_close()
        print("Todas as relações filme-gênero foram inseridas com sucesso.")

    except mysql.connector.Error as err:
        print(f"Erro: {err}")
        conexao.connection.rollback()
    finally:
        conexao.connection.close()


In [ ]:
def capturar_tds_generos():
    conexao = ConexaoBD(config)
    cursor = conexao.cursor
    query = '''
                SELECT * FROM genero
            '''
    cursor.execute(query)
    generos = cursor.fetchall()
    return generos

In [ ]:
def capturar_tds_produtoras():
    conexao = ConexaoBD(config)
    cursor = conexao.cursor
    query = '''
                SELECT * FROM produtora
            '''
    cursor.execute(query)
    produtoras = cursor.fetchall()
    return produtoras

In [ ]:
def capturar_tds_idiomas():
    conexao = ConexaoBD(config)
    cursor = conexao.cursor
    query = '''
                SELECT * FROM idioma
            '''
    cursor.execute(query)
    idiomas = cursor.fetchall()
    return idiomas


In [4]:
def capturar_tds_filmes():
    conexao = ConexaoBD(config)
    cursor = conexao.cursor
    query = '''
                SELECT * FROM filme
            '''
    cursor.execute(query)
    filmes_dados = cursor.fetchall()
    lista_filmes = []

    for filme in filmes_dados:
        # Criar uma instância da classe Filme com os dados obtidos
        filme_instancia = Filme(
            id=filme[0],
            titulo=filme[1],
            media_votos=filme[2],
            contagem_votos=filme[3],
            status_filme=filme[4],
            data_lancamento=filme[5],
            faturamento=filme[6],
            adulto=filme[7],
            orcamento=filme[8],
            id_imdb=filme[9],
            titulo_original=filme[10],
            resumo=filme[11],
            popularidade=filme[12]
        )
        # Adiciona a instância criada à lista de filmes
        lista_filmes.append(filme_instancia)

    conexao.connection.close()
    return lista_filmes

In [3]:
def string_para_lista(entrada):
    # Remover espaços em branco desnecessários e dividir a string pelos separadores de vírgula
    lista = [item.strip() for item in entrada.split(',')]
    return lista


def handle_date(value):
    if pd.notnull(value) and value != '-':
        try:
            date = pd.to_datetime(value, errors='coerce')  # Tentar converter o valor para datetime
            if pd.notnull(date):
                return date.strftime('%Y-%m-%d')  # Retornar no formato esperado pelo MySQL
            else:
                return None
        except ValueError:
            return None
    else:
        return None

def tratar_string(value):
    if pd.isna(value) or value == '' or value is None:
        return "NÃO INFORMADO"
    else:
        return value
    
def tratar_valor(value):
    if pd.isna(value) or value == '' or value is None:
        return None
    else:
        return value
    

